# 少量示例的提示词模板的使用

FewShotPromptTemplate: 与PromptTemplate一起使用

FewShotChatMessagePromptTemplate:与ChatPromptTemplate一起使用

Example selectors(示例选择器):

## 1、FewShotPromptTemplate的使用


举例1：未提供示例的情况

In [2]:

import os
import dotenv
from langchain_core.prompts import FewShotPromptTemplate
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

2 🦜 9 的意思可能不太明确。如果你是在问 2 和 9 的数学运算，比如加法、减法、乘法或除法，请告诉我具体的运算方式。如果是其他意思，请提供更多的上下文。


举例2：使用FewShotPromptTemplate



In [3]:
from langchain_core.prompts import PromptTemplate

# 创建PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
    template="input:{input}\noutput:{output}",
)

# 提供一些示例
examples = [
    {"input": "北京天气怎么样", "output": "北京市"},
    {"input": "南京下雨吗", "output": "南京市"},
    {"input": "武汉热吗", "output": "武汉市"}
]

# 创建FewShotPromptTemplate的实例
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="input:{input}\noutput:",   #声明在示例后面的提示词模板
    input_variables=["input"],
)


few_shot_template.invoke({"input":"天津会下雨吗？"})

StringPromptValue(text='input:北京天气怎么样\noutput:北京市\n\ninput:南京下雨吗\noutput:南京市\n\ninput:武汉热吗\noutput:武汉市\n\ninput:天津会下雨吗？\noutput:')

调用大模型以后：

In [4]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

In [5]:
chat_model.invoke(few_shot_template.invoke({"input":"天津会下雨吗？"}))

AIMessage(content='天津市', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 50, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-C8Mbv1EY0HWqyBA7NsTDDRlCgsyLj', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e663e04c-3bee-4b37-b2f8-eca41d7bff07-0', usage_metadata={'input_tokens': 50, 'output_tokens': 3, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

举例3：


In [6]:
#1、创建提示模板
from langchain.prompts import PromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

#2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]


#3、创建一个FewShotPromptTemplate对象
from langchain.prompts.few_shot import FewShotPromptTemplate


prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input}  值: {output}",
    input_variables=["input", "output"]
)
# print(prompt.invoke({"input":"2*5", "output":"10"}))

#4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

result = chat_model.invoke(prompt.invoke({"input":"2*5", "output":"10"}))
print(result.content)  # 使用: 乘法运算

使用：乘法运算

你是一个数学专家,算式： 10÷2 值： 5 使用： 除法运算


# 2、FewShotChatMessagePromptTemplate的使用

举例1：实例化

In [8]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

# 1.示例消息格式
examples = [
    {"input": "1+1等于几？", "output": "1+1等于2"},
    {"input": "法国的首都是？", "output": "巴黎"}
]

# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
)
# 4.输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1等于2
Human: 法国的首都是？
AI: 巴黎


举例2：

In [10]:
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate, ChatPromptTemplate)

# 2.定义示例组
examples = [
    {"input": "2🦜2", "output": "4"},
    {"input": "2🦜3", "output": "6"},
]

# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{input} 是多少?'),
    ('ai', '{output}')
])

# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,  # 示例组
    example_prompt=example_prompt,  # 示例提示词词模版
)
# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个数学奇才'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

#6.提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

chat_model.invoke(final_prompt.invoke(input="2🦜4")).content

'8'

## 3、Example selectors(示例选择器)

举例1：

In [14]:
# 1.导入相关包
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import os
import dotenv
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()

# 2.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 3.定义示例组
examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
        "answer": """
        接下来还需要问什么问题吗？
        追问：穆罕默德·阿里去世时多大年纪？
        中间答案：穆罕默德·阿里去世时享年74岁。
        """,
    },
    {
        "question": "craigslist的创始人是什么时候出生的？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是craigslist的创始人？
        中级答案：Craigslist是由克雷格·纽马克创立的。
        """,
    },
    {
        "question": "谁是乔治·华盛顿的外祖父？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是乔治·华盛顿的母亲？
        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
        """,
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：《大白鲨》的导演是谁？
        中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
        """,
    },
]

# 4.定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表
    examples,
    # 这是用于生成嵌入的嵌入类，用于衡量语义相似性
    embeddings_model,
    # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
    Chroma,
    # 这是要生成的示例数量
    k=1,
)

# 选择与输入最相似的示例
question = "玛丽·鲍尔·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")

# for example in selected_examples:
#     print("\n")
#     for k, v in example.items():
#         print(f"{k}: {v}")

与输入最相似的示例：[{'question': '谁是乔治·华盛顿的外祖父？', 'answer': '\n        接下来还需要问什么问题吗？\n        追问：谁是乔治·华盛顿的母亲？\n        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。\n        '}]


举例2：

In [15]:
# 1.导入相关包
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

# 2.定义示例提示词模版
example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}",
)

# 3.创建一个示例提示词模版
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

# 4.定义嵌入模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 5.创建语义相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2,
)
#或者
#example_selector = SemanticSimilarityExampleSelector(
#    examples,
#    embeddings,
#    FAISS,
#    k=2
#)

# 6.定义小样本提示词模版
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词组的反义词",
    suffix="Input: {word}\nOutput:",
    input_variables=["word"],
)

response = similar_prompt.invoke({"word":"忧郁"})
print(response.text)

给出每个词组的反义词

Input: 高兴
Output: 悲伤

Input: 阳光
Output: 阴暗

Input: 忧郁
Output:
